# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [99]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats 
import xlsxwriter


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [100]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [101]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data['year1ChangePercent'])



0.019740329777778957


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [102]:
print(data['year1ChangePercent'])

0.019740329777778957


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [103]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 50))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB
AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM
EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS
GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INF

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [104]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try: 
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A'
                    ], index = my_columns
                ), ignore_index = True
            )
        except:
            print(f'failed on {symbol}')
            pass


   

    
    

C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3438664142.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3438664142.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3438664142.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3438664142.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\L

failed on VIAC
failed on WLTW


C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3438664142.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3438664142.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3438664142.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3438664142.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\L

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [105]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,N/A
1,355,OXY,68.77,1.600236,N/A
2,147,DVN,70.34,1.561246,N/A
3,89,CF,106.86,1.364008,N/A
4,315,MRO,25.71,1.222799,N/A
5,109,COP,110.26,1.014337,N/A
6,42,APA,38.8,0.979339,N/A
7,159,EOG,121.52,0.90798,N/A
8,465,VLO,113.82,0.818559,N/A
9,492,XOM,95.59,0.813969,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [106]:
portfolio_size = 100000



In [107]:
position_size = portfolio_size/len(final_dataframe.index)

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe 

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,25
1,355,OXY,68.77,1.600236,29
2,147,DVN,70.34,1.561246,28
3,89,CF,106.86,1.364008,18
4,315,MRO,25.71,1.222799,77
5,109,COP,110.26,1.014337,18
6,42,APA,38.8,0.979339,51
7,159,EOG,121.52,0.90798,16
8,465,VLO,113.82,0.818559,17
9,492,XOM,95.59,0.813969,20


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [142]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]


hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try: 
            hqm_dataframe = hqm_dataframe.append(
                pd.Series(
                    [
                            symbol,
                            data[symbol]['quote']['latestPrice'],
                            'N/A',
                            data[symbol]['stats']['year1ChangePercent'],
                            'N/A',
                            data[symbol]['stats']['month6ChangePercent'],
                            'N/A',
                            data[symbol]['stats']['month3ChangePercent'],
                            'N/A',
                            data[symbol]['stats']['month1ChangePercent'],
                            'N/A',
                            'N/A'
                    ], index = hqm_columns, 
                ), ignore_index = True
            )
        except:
            pass
hqm_dataframe

C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3126122925.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3126122925.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3126122925.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_15648\3126122925.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ip

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,128.01,N/A,-0.276669,N/A,-0.035461,N/A,-0.006686,N/A,-0.024909,N/A,N/A
1,AAL,12.99,N/A,-0.342611,N/A,-0.206475,N/A,-0.256014,N/A,-0.076102,N/A,N/A
2,AAP,171.73,N/A,-0.12312,N/A,-0.165353,N/A,-0.093421,N/A,-0.119424,N/A,N/A
3,AAPL,155.81,N/A,0.01974,N/A,-0.061865,N/A,0.031852,N/A,-0.024895,N/A,N/A
4,ABBV,136.28,N/A,0.266404,N/A,-0.072516,N/A,-0.062659,N/A,-0.029276,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,112.74,N/A,-0.125189,N/A,-0.066769,N/A,-0.058248,N/A,-0.072359,N/A,N/A
497,ZBH,105.57,N/A,-0.287808,N/A,-0.135938,N/A,-0.133214,N/A,-0.074435,N/A,N/A
498,ZBRA,297.6,N/A,-0.494093,N/A,-0.283358,N/A,-0.142536,N/A,-0.07955,N/A,N/A
499,ZION,55.04,N/A,-0.012661,N/A,-0.183803,N/A,-0.02933,N/A,0.04379,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [143]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

#### Before we start, we need to clean some data:
some data has None which will screw up later steps
https://stackoverflow.com/questions/65174575/typeerror-not-supported-between-instances-of-nonetype-and-float


In [144]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

#### Resume
Now we continue with the video and loop through to calculate percentile. 

In [145]:

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,128.01,N/A,-0.276669,0.193613,-0.035461,0.610778,-0.006686,0.706587,-0.024909,0.518962,N/A
1,AAL,12.99,N/A,-0.342611,0.107784,-0.206475,0.209581,-0.256014,0.037924,-0.076102,0.187625,N/A
2,AAP,171.73,N/A,-0.12312,0.479042,-0.165353,0.305389,-0.093421,0.347305,-0.119424,0.071856,N/A
3,AAPL,155.81,N/A,0.01974,0.712575,-0.061865,0.542914,0.031852,0.850299,-0.024895,0.520958,N/A
4,ABBV,136.28,N/A,0.266404,0.898204,-0.072516,0.518962,-0.062659,0.471058,-0.029276,0.499002,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,112.74,N/A,-0.125189,0.469062,-0.066769,0.53493,-0.058248,0.489022,-0.072359,0.211577,N/A
497,ZBH,105.57,N/A,-0.287808,0.173653,-0.135938,0.379242,-0.133214,0.217565,-0.074435,0.195609,N/A
498,ZBRA,297.6,N/A,-0.494093,0.033932,-0.283358,0.091816,-0.142536,0.183633,-0.07955,0.179641,N/A
499,ZION,55.04,N/A,-0.012661,0.640719,-0.183803,0.251497,-0.02933,0.59481,0.04379,0.89022,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [146]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time in time_period: 
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)


hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,128.01,N/A,-0.276669,0.193613,-0.035461,0.610778,-0.006686,0.706587,-0.024909,0.518962,0.518962
1,AAL,12.99,N/A,-0.342611,0.107784,-0.206475,0.209581,-0.256014,0.037924,-0.076102,0.187625,0.187625
2,AAP,171.73,N/A,-0.12312,0.479042,-0.165353,0.305389,-0.093421,0.347305,-0.119424,0.071856,0.071856
3,AAPL,155.81,N/A,0.01974,0.712575,-0.061865,0.542914,0.031852,0.850299,-0.024895,0.520958,0.520958
4,ABBV,136.28,N/A,0.266404,0.898204,-0.072516,0.518962,-0.062659,0.471058,-0.029276,0.499002,0.499002
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,112.74,N/A,-0.125189,0.469062,-0.066769,0.53493,-0.058248,0.489022,-0.072359,0.211577,0.211577
497,ZBH,105.57,N/A,-0.287808,0.173653,-0.135938,0.379242,-0.133214,0.217565,-0.074435,0.195609,0.195609
498,ZBRA,297.6,N/A,-0.494093,0.033932,-0.283358,0.091816,-0.142536,0.183633,-0.07955,0.179641,0.179641
499,ZION,55.04,N/A,-0.012661,0.640719,-0.183803,0.251497,-0.02933,0.59481,0.04379,0.89022,0.89022


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [147]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,COG,22.25,N/A,0.340483,0.934132,0.180873,0.948104,0.255629,0.994012,0.249298,1.0,1.0
1,UNM,37.63,N/A,0.480773,0.956088,0.354118,0.988024,0.037385,0.866267,0.199936,0.998004,0.998004
2,CAH,70.99,N/A,0.374912,0.94012,0.351797,0.986028,0.314708,0.996008,0.187918,0.996008,0.996008
3,AES,26.15,N/A,0.096091,0.796407,0.239395,0.968064,0.189556,0.988024,0.176338,0.994012,0.994012
4,COP,110.26,N/A,1.014337,0.99002,0.146996,0.924152,-0.029095,0.598802,0.169421,0.992016,0.992016
5,NLSN,27.85,N/A,0.327221,0.928144,0.571706,0.996008,0.087288,0.934132,0.163432,0.99002,0.99002
6,DVN,70.34,N/A,1.561246,0.996008,0.239675,0.97006,-0.062947,0.469062,0.160152,0.988024,0.988024
7,EOG,121.52,N/A,0.90798,0.986028,0.078967,0.844311,-0.109816,0.289421,0.133688,0.986028,0.986028
8,HRB,44.47,N/A,0.767853,0.974052,0.733048,0.998004,0.23744,0.992016,0.131552,0.984032,0.984032
9,PFG,74.44,N/A,0.153025,0.838323,0.09337,0.866267,0.031168,0.846307,0.124641,0.982036,0.982036


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [148]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
position_size 


for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,COG,22.25,89,0.340483,0.934132,0.180873,0.948104,0.255629,0.994012,0.249298,1.0,1.0
1,UNM,37.63,53,0.480773,0.956088,0.354118,0.988024,0.037385,0.866267,0.199936,0.998004,0.998004
2,CAH,70.99,28,0.374912,0.94012,0.351797,0.986028,0.314708,0.996008,0.187918,0.996008,0.996008
3,AES,26.15,76,0.096091,0.796407,0.239395,0.968064,0.189556,0.988024,0.176338,0.994012,0.994012
4,COP,110.26,18,1.014337,0.99002,0.146996,0.924152,-0.029095,0.598802,0.169421,0.992016,0.992016
5,NLSN,27.85,71,0.327221,0.928144,0.571706,0.996008,0.087288,0.934132,0.163432,0.99002,0.99002
6,DVN,70.34,28,1.561246,0.996008,0.239675,0.97006,-0.062947,0.469062,0.160152,0.988024,0.988024
7,EOG,121.52,16,0.90798,0.986028,0.078967,0.844311,-0.109816,0.289421,0.133688,0.986028,0.986028
8,HRB,44.47,44,0.767853,0.974052,0.733048,0.998004,0.23744,0.992016,0.131552,0.984032,0.984032
9,PFG,74.44,26,0.153025,0.838323,0.09337,0.866267,0.031168,0.846307,0.124641,0.982036,0.982036


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [149]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)


## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [150]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [151]:
column_formats = {
    'A': ['Ticker', string_template], 
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])



## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [152]:

writer.save()